In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

np.seterr('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 코사인 유사도를 이용한 영화 추천 알고리즘 (item-based recommendation)

###### 데이터를 불러오기 위한 경로 지정 및 함수생성
생성 데이터: tags, ratings, movies, links

In [ ]:
import sys
mod = sys.modules[__name__]

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.split('.')[1] == 'csv':
            name = filename.split('.')[0]
            data_path = os.path.join(dirname, filename)
            setattr(mod, name, pd.read_csv(data_path))      

print("Finish Load Data!!")

## 1.1. 데이터 설명
This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files links.csv, movies.csv, ratings.csv and tags.csv. More details about the contents and use of all these files follows.

This is a development dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available benchmark datasets if that is your intent.

This and other GroupLens data sets are publicly available for download at

Summary
=======

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/>.


Usage License
=============

Neither the University of Minnesota nor any of the researchers involved can guarantee the correctness of the data, its suitability for any particular purpose, or the validity of results based on the use of the data set. The data set may be used for any research purposes under the following conditions:

* The user may not state or imply any endorsement from the University of Minnesota or the GroupLens Research Group.
* The user must acknowledge the use of the data set in publications resulting from the use of the data set (see below for citation information).
* The user may redistribute the data set, including transformations, so long as it is distributed under these same license conditions.
* The user may not use this information for any commercial or revenue-bearing purposes without first obtaining permission from a faculty member of the GroupLens Research Project at the University of Minnesota.
* The executable software scripts are provided "as is" without warranty of any kind, either expressed or implied, including, but not limited to, the implied warranties of merchantability and fitness for a particular purpose. The entire risk as to the quality and performance of them is with you. Should the program prove defective, you assume the cost of all necessary servicing, repair or correction.

In no event shall the University of Minnesota, its affiliates or employees be liable to you for any damages arising out of the use or inability to use these programs (including but not limited to loss of data or data being rendered inaccurate).

If you have any further questions or comments, please email <grouplens-info@umn.edu>


Citation
========

To acknowledge use of the dataset in publications, please cite the following paper:

> F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>


Further Information About GroupLens
===================================

GroupLens is a research group in the Department of Computer Science and Engineering at the University of Minnesota. Since its inception in 1992, GroupLens's research projects have explored a variety of fields including:

* recommender systems
* online communities
* mobile and ubiquitious technologies
* digital libraries
* local geographic information systems

GroupLens Research operates a movie recommender based on collaborative filtering, MovieLens, which is the source of these data. We encourage you to visit <http://movielens.org> to try it out! If you have exciting ideas for experimental work to conduct on MovieLens, send us an email at <grouplens-info@cs.umn.edu> - we are always interested in working with external collaborators.


Content and Use of Files
========================

Formatting and Encoding
-----------------------

The dataset files are written as [comma-separated values](http://en.wikipedia.org/wiki/Comma-separated_values) files with a single header row. Columns that contain commas (`,`) are escaped using double-quotes (`"`). These files are encoded as UTF-8. If accented characters in movie titles or tag values (e.g. Misérables, Les (1995)) display incorrectly, make sure that any program reading the data, such as a text editor, terminal, or script, is configured for UTF-8.


User Ids
--------

MovieLens users were selected at random for inclusion. Their ids have been anonymized. User ids are consistent between `ratings.csv` and `tags.csv` (i.e., the same id refers to the same user across the two files).


Movie Ids
---------

Only movies with at least one rating or tag are included in the dataset. These movie ids are consistent with those used on the MovieLens web site (e.g., id `1` corresponds to the URL <https://movielens.org/movies/1>). Movie ids are consistent between `ratings.csv`, `tags.csv`, `movies.csv`, and `links.csv` (i.e., the same id refers to the same movie across these four data files).


Ratings Data File Structure (ratings.csv)
-----------------------------------------

All ratings are contained in the file `ratings.csv`. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

    userId,movieId,rating,timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.


Tags Data File Structure (tags.csv)
-----------------------------------

All tags are contained in the file `tags.csv`. Each line of this file after the header row represents one tag applied to one movie by one user, and has the following format:

    userId,movieId,tag,timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user.

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.


Movies Data File Structure (movies.csv)
---------------------------------------

Movie information is contained in the file `movies.csv`. Each line of this file after the header row represents one movie, and has the following format:

    movieId,title,genres

Movie titles are entered manually or imported from <https://www.themoviedb.org/>, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.

Genres are a pipe-separated list, and are selected from the following:

* Action
* Adventure
* Animation
* Children's
* Comedy
* Crime
* Documentary
* Drama
* Fantasy
* Film-Noir
* Horror
* Musical
* Mystery
* Romance
* Sci-Fi
* Thriller
* War
* Western
* (no genres listed)


Links Data File Structure (links.csv)
---------------------------------------

Identifiers that can be used to link to other sources of movie data are contained in the file `links.csv`. Each line of this file after the header row represents one movie, and has the following format:

    movieId,imdbId,tmdbId

movieId is an identifier for movies used by <https://movielens.org>. E.g., the movie Toy Story has the link <https://movielens.org/movies/1>.

imdbId is an identifier for movies used by <http://www.imdb.com>. E.g., the movie Toy Story has the link <http://www.imdb.com/title/tt0114709/>.

tmdbId is an identifier for movies used by <https://www.themoviedb.org>. E.g., the movie Toy Story has the link <https://www.themoviedb.org/movie/862>.

Use of the resources listed above is subject to the terms of each provider.


Cross-Validation
----------------

Prior versions of the MovieLens dataset included either pre-computed cross-folds or scripts to perform this computation. We no longer bundle either of these features with the dataset, since most modern toolkits provide this as a built-in feature. If you wish to learn about standard approaches to cross-fold computation in the context of recommender systems evaluation, see [LensKit](http://lenskit.org) for tools, documentation, and open-source code examples.

In [ ]:
movies

In [ ]:
ratings

In [ ]:
tags

In [ ]:
links

In [ ]:
def weight_rating(ratings, p = 0.95):
    """
    R: average for the movie (mean) = (Rating)
    v = number of votes for the movie = (votes)
    m = minimum votes requred to be list in the top 5%
    C = the mean vote across the whole report
    """
    rating_data = ratings.copy()
    rating_data['count'] = 1
    rating_data = rating_data.pivot_table(index = 'movieId', aggfunc = {'rating': 'mean', 'count': 'sum'})

    m = rating_data['count'].quantile(p)
    rating_data = rating_data[rating_data['count'] >= m]

    R = rating_data['rating']
    v = rating_data['count']

    C = rating_data['rating'].mean()
    
    rating_data['weight_rating'] = ((v / (v + m)) * R) + ((m / (v + m)) * C)
    
    return rating_data

In [ ]:
rating_data = weight_rating(ratings, p = 0.95)

In [ ]:
rating_data['weight_rating'].describe()

In [ ]:
fig, ax1 = plt.subplots(nrows = 1, ncols = 1, figsize = (25, 10))
rating_data['rating'].plot(kind = 'kde', ax = ax1)
rating_data['weight_rating'].plot(kind = 'kde', color = 'r', ax = ax1)
fig.suptitle('Distribution of original rating and weight rating', fontsize = 18)
plt.legend(loc = 'best', fontsize = 15)

In [ ]:
data = pd.merge(left = movies.set_index(['movieId']), right = rating_data, how = 'inner', left_index = True, right_index = True)

In [ ]:
data

In [ ]:
data['genres']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer(ngram_range = (1,3))

In [ ]:
c_vector_genres = count_vector.fit_transform(data['genres'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
genres_similarity = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:,::-1]

In [ ]:
if 'title' in data.columns:
    data = data.set_index(['title'])
list_movie = data.index

In [ ]:
movie = 'Toy Story (1995)'

In [ ]:
def recommend_movie_by_similar_movie(movie, n_recommend = 3, genres_similarity = genres_similarity, data = data):
    """
    movie: 추천 받고자하는 movie (ex: 'Toy Story (1995)')
    n_recommend: 추천받을 영화의 수
    genres_similarity: genres에 대한 cosine similarity matrix
    data: moive의 index를 위한 데이터
    """
    # pick movie index
    movie_index = list(data.index).index(movie)
    
    # extract movie's cosine similarity matrix
    movie_genres_similarity = genres_similarity[movie_index, :(n_recommend +1)].reshape(-1)
    movie_genres_similarity = [ix for ix in movie_genres_similarity if ix != movie_index]
    
    # convert recommend movie list
    recommend_movies = [data.index[ix] for ix in movie_genres_similarity]
    
    print('Recommend with %s' %(movie))
    print(recommend_movies)

    return recommend_movies

In [ ]:
recommend_movie_by_similar_movie(movie = 'Guardians of the Galaxy (2014)', n_recommend = 30)